# Kings County Housing Prices Bakeoff

Below are a list of steps that you should take while trying to complete your bake-off entry.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# for looking at models r-squared, coefficients, and p-values
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
# used to change date feature
from datetime import datetime
from dateutil.parser import parse
# for stats tests
from scipy.stats import t
from scipy import stats
# used to get subset of overall dataset
from scipy.stats.mstats import winsorize
# used to get train test split for model testing
from sklearn.model_selection import train_test_split
# for feature selection
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
# for feature selection
from sklearn.feature_selection import RFECV
import warnings
warnings.filterwarnings('ignore')

# Read in Data

In [ ]:
df = pd.read_csv('kc_house_data_train.csv')

# Exploratory Data Analysis 

In [ ]:
# removing unnecessary column, provides no info
df = df.drop(['Unnamed: 0','id'], axis=1)
pd.options.display.max_columns = None

In [ ]:
# Looking at the data itself
df.head()

In [ ]:
# checking to see if any major outliers
df.describe()

In [ ]:
# looking to see if any features correlate with price
df.corr()['price'].sort_values(ascending=False)

In [ ]:
# looking at regression plots to see if any trends are immediatly present
x_sqft = df.sqft_living
x_grade = df.grade
x_sqft15 = df.sqft_living15
x_view = df.view
y_price_test = df.price

fig, ax = plt.subplots(2,2,figsize=(10,10))
ax[0,0].scatter(x_sqft,y_price_test,color='red')
ax[0,0].set_xlabel('sqft_living')
ax[0,0].set_ylabel('price')
ax[0,1].scatter(x_grade,y_price_test,color='pink')
ax[0,1].set_xlabel('grade')
ax[0,1].set_ylabel('price')
ax[1,0].scatter(x_sqft15,y_price_test,color='tomato')
ax[1,0].set_xlabel('sqft_living15')
ax[1,0].set_ylabel('price')
ax[1,1].scatter(x_view,y_price_test,color='firebrick')
ax[1,1].set_xlabel('view')
ax[1,1].set_ylabel('price')

fig.savefig('scatter_for_git')

In [ ]:
# Left commented due to the time it takes to run this pairplot
# was very helpful to identify categorical variables
#sns.pairplot(df)

In [ ]:
# regression plot to see the correlation
sns.regplot('sqft_living','price',data=df,color='green',line_kws={"color": "red"})

In [ ]:
# regression plot to see the correlation
sns.regplot('grade','price',data=df,color='darkorange',line_kws={"color": "navy"})

# Cleaning up any issues (extreme values, etc.) with the data.  

Remember that you can't just delete rows with extreme values. Similar observations might be present in the holdout data set, and you can't just delete those rows and not have a prediction for it. 

In [ ]:
df.describe()

In [ ]:
"""
OUTLIERS
- bed and bath seem to have some extreme values
- sqft has a large outlier
- sqft 15 has outlier but assuming fancy neighbor hood
"""

In [ ]:
a = np.array([10, 4, 9, 8, 5, 3, 7, 2, 1, 6])
winsorize(a, limits=[0.1, 0.1],inclusive=(False,True))

# Generating New Features

First I am going to change the date column to a more usable datetime object.

### New Feature: Month Sold

Going to look to see if the month the house was sold has any affect on the price it was sold for.

In [ ]:
# converting date to a real datetime
df.date = pd.to_datetime(df['date'])
# extracting just the month
df['month_sold'] = df.date.dt.month
# grouping to see price per month
selling_month = df.groupby(['month_sold']).mean()['price']

In [ ]:
# quick visualization to see if anything sticks out
x = selling_month.index
y = selling_month.values
fig = plt.figure(figsize = (6,4))
ax = fig.add_subplot()
ax.bar(x,y,color='salmon')
ax.set_title('Month Sold vs Price')
ax.set_xlabel('Month Sold')
ax.set_ylabel('Price')

From this plot we can see that there is a variation in price based on the month it was sold. The difference between the lowest and the highest is only about $6,000.

### New Feature: Year Since Renovation/Build

I wanted to look to see if the time since the house was last imporoved or built has any affect on the price of the house.

In [ ]:
# using np.select to find the years since it was built or renovated
# if it was renovated
conditions = [
    df['yr_renovated'] != 0,
]
# set the years since build to 2020 - that year to get the # of years
# the data stops at 2015 but all of these will be changed so the time since does not matter
choices = [
    2020-df['yr_renovated']
]
# if not renovated defaults to the year it was built
df['yr_since_build'] = np.select(conditions,choices,default=(2020-df['yr_built']))

In [ ]:
# using scatter to see if any trends stick out
x = df['yr_since_build']
y = df['price']
fig = plt.figure(figsize = (6,4))
ax = fig.add_subplot()
ax.scatter(x,y,color='purple')
ax.set_title('Year Since Built vs Price ')
ax.set_xlabel('Year Since Build')
ax.set_ylabel('Price')

### New Feature: Is Multi Floor Home

This feature is going to be created because I want to see if there is any major price increase for houses that have multiple floors.

In [ ]:
# looking at the counts of floors per home
df.floors.value_counts()

In [ ]:
# using lambda to see if floors is greater than 1
df['is_multi_floor'] = df['floors'].apply(lambda x: 1 if x > 1 else 0)

In [ ]:
# totally even split of 
df.is_multi_floor.value_counts()

In [ ]:
# looking at differences in prices
df.groupby('is_multi_floor').mean()['price']

In [ ]:
plt.figure(figsize=(5,10))
sns.scatterplot(df.grade,df.price,hue=df.is_multi_floor)

### New Feature: If House has new basement

In [ ]:
# using lambda to see if there is a square footage for the beasement
df['has_basement'] = df['sqft_basement'].apply(lambda x: 1 if x>0 else 0)

In [ ]:
# looking at counts to see the spread
df['has_basement'].value_counts()

In [ ]:
# see if the prices differ
df.groupby('has_basement').mean()['price']

## Finalizing feature dataframe

In [ ]:
features = [
    'bedrooms',
    'bathrooms',
    'sqft_living',
    'sqft_lot',
    'floors',
    'waterfront',
    'view',
    'condition',
    'grade',
    'sqft_above',
    'sqft_basement',
    'yr_built',
    'yr_renovated',
    'sqft_living15',
    'sqft_lot15',
    'month_sold',
    'yr_since_build',
    'is_multi_floor',
    'has_basement'
]

In [ ]:
df_features = df[features]
target = df['price']

### Creating polynomial feature from the grade

Due to the look of the graph that we saw for grade, I thought that a polynomial feature could better describe that output. I was doing this manually but this will be encorporated in the larger polynomial combinations we make after.

In [ ]:
# getting the data I need out
tdf = df[['price','grade']]
x = tdf.drop(columns='price',axis=1)
y = tdf['price']
print(len(x),len(y))

In [ ]:
plt.scatter(x, y, color='green')
plt.xlabel('grade')
plt.ylabel('price')

In [ ]:
from sklearn.linear_model import LinearRegression
#x.reshape(-1,1)
reg = LinearRegression().fit(x, y)

In [ ]:
plt.scatter(x, y, color='green')
plt.plot(x, reg.predict(x))
plt.xlabel('view')
plt.ylabel('price');

In [ ]:
x['grade_square'] = x.grade**2

In [ ]:
reg = LinearRegression().fit(x, y)

In [ ]:
plt.scatter(x.grade, y, color='green')
plt.plot(x['grade'], reg.predict(x))
plt.xlabel('grade')
plt.ylabel('price');

## Making Polynomial Features

This will allow us to make combinations of all features up to a certain degree. I am going to make these now and test later.

### Creation of new dataframe for polynomials of degree 2

In [ ]:
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

In [ ]:
# instantiating the object
poly2 = PolynomialFeatures(degree=2, include_bias=False)
# transforming my features
poly2_data = poly2.fit_transform(df_features)
# creating the new data frame
poly2_cols = poly2.get_feature_names(df_features.columns)
df_poly2 = pd.DataFrame(poly2_data,columns = poly2_cols)
df_poly2.head()

In [ ]:
df_poly2.shape

### Creation of new dataframe for polynomials of degree 3

In [ ]:
# instantiating the object
poly3 = PolynomialFeatures(degree=3, include_bias=False)
# transforming my features
poly3_data = poly3.fit_transform(df_features)
# creating the new data frame
poly3_cols = poly3.get_feature_names(df_features.columns)
df_poly3 = pd.DataFrame(poly3_data,columns = poly3_cols)
df_poly3.head()

In [ ]:
df_poly3.shape

## Statistical Tests

#### Number 1
* Ho - There is no difference in price between waterfront and not
* Ha - There is a difference in price

#### Number 2
* Ho - A basement will not increase the price of a home
* Ha - A basement will increase a price of a home

#### Number 3
* Ho - a house is the same sqft as its 15 neighbors
* Ha - a house is not the same as the 15 neighbors

### Number 1

In [ ]:
# going to use a two sample t-test
# getting the two samples
water = df[df['waterfront'] > 0.5]
land = df[(df.waterfront < 0.5)]
# degrees of freedom with this case
dfree = len(water) + len(land) - 1
# going for a 5% alpha with 2.5% on each side
a = 0.025
value= t.ppf(a, dfree)
print('the critical value is '+str(value))
p = t.cdf(value,dfree)
print('the p value is '+str(p))
stats.ttest_ind(water['price'], land['price'])

**We can reject the null hypothesis so we can see that waterfront definitly does change the price**

### Number 2

In [ ]:
basement = df[df['has_basement'] == 1]
nobase = df[df['has_basement'] == 0]
dfree = len(basement) + len(nobase) - 2
a = 0.05
value= t.ppf(a, dfree)
print('the critical value is '+str(value))
p = t.cdf(value,dfree)
print('the p value is '+str(p))
stats.ttest_ind(basement['price'], nobase['price'])


**We can reject the null hypothesis so we can see that a basement does increase the price**

### Number 3

In [ ]:
# going to use a two-tailed t-test to tell if these two groups are the same or different
my_house = df.sqft_living
neighbors = df.sqft_living15

a = 0.025
dfree = len(my_house) + len(neighbors) - 2

value= t.ppf(a, dfree)
print('the critical value is '+str(value))
p = t.cdf(value,dfree)
print('the p value is '+str(p))
stats.ttest_ind(my_house, neighbors)

**We can reject the null hypothesis so we can say that with 95% confidence our house is likely to be different in square footage than our neighbors.**

# Creating the Train-Test Split 

### Perform a train-test split of the data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_features, target, random_state=9,test_size=0.2)
print(len(X_train), "train +", len(X_test), "test")

### Baseline Linear Regression

In [ ]:
scaler = StandardScaler()
# fit the scaler to the training data
scaler.fit(X_train)
# transforming the training data
X_train = pd.DataFrame(data=scaler.transform(X_train), columns = df_features.columns)
# transforming the test data seperatley
X_test = pd.DataFrame(data=scaler.transform(X_test), columns = df_features.columns)

In [ ]:
# testing this data on a linear regression
LinReg = LinearRegression()
# fitting our object to the data
LinReg = LinReg.fit(X_train,y_train)
# making predictions on our y_train
y_train_pred = LinReg.predict(X_train)
# getting the RSME
train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
print('Training Root Mean Squared Error:' , train_rmse)

In [ ]:
# evaluating what we have done on the test set
# using the fitted model to predict on our test (holdout)
y_test_pred = LinReg.predict(X_test)
# getting rsme
test_rmse = np.sqrt(metrics.mean_squared_error(y_test,y_test_pred))
print('Testing Root Mean Squared Error:' , test_rmse)
print('Training: ', int(train_rmse), "vs. Testing: ", int(test_rmse))                      

## Testing polynomials of degree 2 for RSME

In [ ]:
# creating a new test train split for polynomials
X_train, X_test, y_train, y_test = train_test_split(df_poly2, target, random_state=9,test_size=0.2)

In [ ]:
# instantiating a scaler
scaler_p2 = StandardScaler()
# fit the scaler to the training data
scaler_p2.fit(X_train)
# transforming the training data
X_train = pd.DataFrame(data=scaler_p2.transform(X_train), columns = df_poly2.columns)
# transforming the test data seperatley
X_test = pd.DataFrame(data=scaler_p2.transform(X_test), columns = df_poly2.columns)

In [ ]:
# testing this data on a linear regression
LinReg_p2 = LinearRegression()
# fitting our object to the data
LinReg_p2 = LinReg.fit(X_train,y_train)
# making predictions on our y_train
y_train_pred = LinReg_p2.predict(X_train)
# getting the RSME
train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
print('Training Root Mean Squared Error:' , train_rmse)

In [ ]:
# evaluating what we have done on the test set
# using the fitted model to predict on our test (holdout)
y_test_pred = LinReg_p2.predict(X_test)
# getting rsme
test_rmse = np.sqrt(metrics.mean_squared_error(y_test,y_test_pred))
print('Testing Root Mean Squared Error:' , test_rmse)
print('Training: ', int(train_rmse), "vs. Testing: ", int(test_rmse))                      

## Testing polynomials of degree 3 for RSME

In [ ]:
# creating a new test train split for polynomials
X_train, X_test, y_train, y_test = train_test_split(df_poly3, target, random_state=4,test_size=0.2)


In [ ]:
# instantiating a scaler
scaler_p3 = StandardScaler()
# fit the scaler to the training data
scaler_p3.fit(X_train)
# transforming the training data
X_train = pd.DataFrame(data=scaler_p3.transform(X_train), columns = df_poly3.columns)
# transforming the test data seperatley
X_test = pd.DataFrame(data=scaler_p3.transform(X_test), columns = df_poly3.columns)

In [ ]:
# testing this data on a linear regression
LinReg_p3 = LinearRegression()
# fitting our object to the data
LinReg_p3 = LinReg.fit(X_train,y_train)
# making predictions on our y_train
y_train_pred = LinReg_p3.predict(X_train)
# getting the RSME
train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
print('Training Root Mean Squared Error:' , train_rmse)

In [ ]:
# evaluating what we have done on the test set
# using the fitted model to predict on our test (holdout)
y_test_pred = LinReg_p3.predict(X_test)
# getting rsme
test_rmse = np.sqrt(metrics.mean_squared_error(y_test,y_test_pred))
print('Testing Root Mean Squared Error:' , test_rmse)
print('Training: ', int(train_rmse), "vs. Testing: ", int(test_rmse))                      

This model is VERY overfit and needs to be run through a feature selection process in order to determine its usefullness.

## Feature Selection

### Filter Method Using Select K-Best

In [ ]:
# using poly_2 as it was the best performing and not overfit
X_train, X_test, y_train, y_test = train_test_split(df_poly2, target, random_state=9,test_size=0.2)

\* I am using poly_2 as it performed the best after doing feature seleciton and with so many features the wrapper method took too long.

In [ ]:
X_train.shape

In [ ]:
# number of features I want to end with
k = round(np.sqrt(len(df_poly2)))
k

### f_regression as ranking method

In [ ]:
# instantiating a feature selector object
feature_selector = SelectKBest(f_regression,131)
# fitting to our data
feature_selector.fit(X_train,y_train)

In [ ]:
# features that we kepy
selected_features = X_train.columns[feature_selector.get_support()]
len(selected_features)

In [ ]:
# Test that for the regression
# instantiate a linear regression object
LR_kbest_F = LinearRegression()
# fit the linear regression to the data
LR_kbest_F = LR_kbest_F.fit(X_train[selected_features], y_train)
# predicting on the data
y_train_kbest_F = LR_kbest_F.predict(X_train[selected_features])
# getting the rsme
train_kbest_F_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_kbest_F))
print('Training Root Mean Squared Error:' , train_kbest_F_rmse)
# predicting on test set
y_kbest_F = LR_kbest_F.predict(X_test[selected_features])
# getting RSME
test_kbest_F_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_kbest_F))
print('Testing Root Mean Squared Error:' , test_kbest_F_rmse)

As we can see this performed much worse than having all features did.

### Mutual Information as ranking method

In [ ]:
# instantiating a feature selector object
feature_selector = SelectKBest(mutual_info_regression, k)
# fitting to our data
feature_selector.fit(X_train,y_train)

In [ ]:
# features that we kepy
selected_features_MI = X_train.columns[feature_selector.get_support()]
len(selected_features_MI)

In [ ]:
# Test that for the regression
# instantiate a linear regression object
LinReg_kbestMI = LinearRegression()
# fit the linear regression to the data
LinReg_kbestMI = LinReg_kbestMI.fit(X_train[selected_features], y_train)
# predicting on the data
y_train_kbestMI = LinReg_kbestMI.predict(X_train[selected_features])
# getting the rsme
trainKMI_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_kbestMI))
print('Training Root Mean Squared Error:' , trainKMI_rmse)
# predicting on test set
y_kbestMI = LinReg_kbestMI.predict(X_test[selected_features])
# getting RSME
testKMI_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_kbestMI))
print('Testing Root Mean Squared Error:' , testKMI_rmse)

### Wrapper Method

In [ ]:
# importing ols model as our esimation of 'goodness'
ols = LinearRegression()
# creating a selector object
feature_selector = RFECV(estimator=ols, step=1, cv=5, scoring='neg_mean_squared_error',n_jobs=-1)
# fitting to our data
feature_selector.fit(X_train, y_train)

In [ ]:
selected_wrapper = X_train.columns[feature_selector.support_]

In [ ]:
len(selected_wrapper)

In [ ]:
# Test that for the regression
# instantiate a linear regression object
LR_wrapper = LinearRegression()
# fit the linear regression to the data
LR_wrapper = LR_wrapper.fit(X_train[selected_wrapper], y_train)
# predicting on the data
y_train_wrapper = LR_wrapper.predict(X_train[selected_wrapper])
# getting the rsme
train_w_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_wrapper))
print('Training Root Mean Squared Error:' , trainW_rmse)
# predicting on test set
y_wrapper_pred = LR_wrapper.predict(X_test[selected_wrapper])
# getting RSME
test_W_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_wrapper_pred))
print('Testing Root Mean Squared Error:' , test_W_rmse)

## Looking at which model performed the best

In [ ]:
print('K-Best F-Regression RMSE:' , testKF_rmse)
print('K-Best Mutual Info RMSE:' , testKMI_rmse)
print('Wrapper RMSE:' , testW_rmse)

Based on these results our wrapper method performs the best.

## Refiting best model to the dataset

In [ ]:
final_df = df_poly2

In [ ]:
# estimator for the wrapper method
ols = LinearRegression()
# final feature selector
feature_selector = RFECV(estimator=ols, step=1, cv=5, scoring='neg_mean_squared_error',n_jobs=-1)
# fitting to the final df
feature_selector.fit(df_poly2,target)
# getting the selected features
selected_features_final = final_df.columns[feature_selector.get_support()]

In [ ]:
list(selected_features_final)

In [ ]:
final_df.shape

In [ ]:
LinReg_final = LinearRegression()
# fitting to the whole dataset
LinReg_final = LinReg_final.fit(df_poly2[selected_features_final],target)
# getting coeficients
LinReg_final.coef_

In [ ]:
print ("R^2 Score:", LinReg_final.score(final_df[selected_features_final], target))

The r-squared looks quite good accounting for 76% of errors.

In [ ]:
# making our final predicitons
y_pred_final = LinReg_final.predict(df_poly2[selected_features_final])

In [ ]:
train_mae = metrics.mean_absolute_error(y_train, y_train_pred)
train_mse = metrics.mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))


print('Mean Absolute Error:', train_mae )
print('Mean Squared Error:',  train_mse)
print('Root Mean Squared Error:' , train_rmse)

## Model Evaluation

In [ ]:
print ("R^2 Score:", LinReg_final.score(df_poly2[selected_features_final],target))

The R-squared seems quite good accounting for almost 76% of error.

In [ ]:
#y_pred = LinReg_final.predict(df_poly2[selected_features_final])

In [ ]:
#import the metrics module from sklearn
from sklearn import metrics

train_mae = metrics.mean_absolute_error(target, y_pred_final)
train_mse = metrics.mean_squared_error(target, y_pred_final)
train_rmse = np.sqrt(metrics.mean_squared_error(target, y_pred_final))


print('Mean Absolute Error:', train_mae )
print('Mean Squared Error:',  train_mse)
print('Root Mean Squared Error:' , train_rmse)

From this we can see that the model performed quite well.

Going to look if our residuals are evenly distributed.

In [ ]:
residuals = (target - y_pred_final)

In [ ]:
plt.hist(residuals,bins=50)

This does look normally distributed but the density in the middle leeds me to believe it is leptokurtic.

Looking to see if our errors are IID and homoscedastic.

In [ ]:
sns.residplot(y_pred_final, target, lowess=True, color="g")

This is somewhat concerning becuase I do see a downward trend.

## Model Interpretation

In [ ]:
for item in enumerate(zip(selected_features_final,LinReg_final.coef_)):
    print('{}. {}: {}'.format(item[0]+1,item[1][0],item[1][1]))

Looking at our selected features above we can see that there are some key feautres that played a part in the model.

`sqft_living` was part of most of the features that we ended up with and this makes a lot of sense. As a house gets bigger it is going to be more expensive. Not only does this mean that it was more expensive to build but that it most likely sits on a larger property or has multiple stories.

`bedrooms` and `bathrooms` also were a part of many of the features selected. This makes sense because as a house gets bigger and is being built to fit more people you are going to need more bedrooms and bathrooms.

`grade` was a part of many of the interactions made in the polynomial generation. This makes sense as the 'better' quality that the house is evaluated to be the more expensive it will be.

As for the coeficients we can see that some are positive and some are negative. I take this as the fine tuning that the model did to interpret how much a houses value would go up and down based on the value of that feature. Due to the scaler that we used we are evalutating these features as their affect on a house price based on standard deviations changes in the feature. 

## Pickling

https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/

In [ ]:
import pickle

pickle_out = open('model.pickle','wb')
pickle.dump(LinReg_final, pickle_out)
pickle_out.close()

In [ ]:
pickle_out = open('scaler.pickle','wb')
pickle.dump(scaler, pickle_out)
pickle_out.close()